# Installs

In [ ]:
pip install pymongo==3.11.0

# Imports

In [ ]:
from os import urandom
from pprint import pprint as pp
from pymongo import MongoClient

# Database Link

In [ ]:
client = MongoClient("database", 27017)
database = client.polymorph
collection = database["clients"]

# Model

In [ ]:
class Client():

    def __init__(self, client):

        self.email = client["email"]
        self.password = client["password"]

        self.id = client["id"] if "id" in client else None

        if "settings" in client:

            self.settings = client["settings"]

        else:

            self.settings = {
                                "general": {
                                    "language": "en-US",
                                    "scale": "metric",
                                    "unit": {"imperial": "inch", "metric": "millimeter"},
                                    "snap": False,
                                    "size": 100,
                                    "grid": 1
                                },
                                "ui": {
                                    "title": "POLYMORPH",
                                    "navbar": True,
                                    "forkme": False,
                                    "metabox": True,
                                    "help": False,
                                    "shortcuts": ["box", "cylinder", "sphere"]
                                },
                                "tooltips": {
                                    "boundingBox": True,
                                    "guidelines": True,
                                    "measurements": False,
                                    "rotation": True
                                },
                                "panels": {
                                    "camera": True,
                                    "lights": False,
                                    "menu": False,
                                    "meshes": True,
                                    "settings": False,
                                    "shapes": False,
                                    "shortcuts": True
                                },
                                "axes": {
                                    "axesCaps": True,
                                    "xAxis": True,
                                    "yAxis": True,
                                    "zAxis": True,
                                    "xyPlane": False,
                                    "xzPlane": False,
                                    "yzPlane": False,
                                    "xyPlaneStep": 10,
                                    "xzPlaneStep": 10,
                                    "yzPlaneStep": 10
                                },
                                "camera": {
                                    "position": {"x": 135, "y": 135, "z": 55},
                                    "target": {"x": 0, "y": 0, "z": 0},
                                    "open": "speed"
                                },
                                "controls": {
                                    "dragSpeed": 50,
                                    "flySpeed": 50,
                                    "zoomSpeed": 50,
                                    "zoomMax": 500,
                                    "zoomMin": 1
                                }
                            }

    def refresh_id(self):

        self.id = urandom(42).hex()

        return self

    def update_settings(self, category, setting, value):

        self.settings[category][setting] = value

        return self

# Basic Functions

In [ ]:
def new_client(client):

    return str(collection.insert_one(client).inserted_id)

def new_clients(clients):

    return str(collection.insert_many(clients).inserted_ids)

def find_client(query={}, filter={"_id": 0}):

    return dict(collection.find_one(query, filter))

def find_clients(query={}, filter={"_id": 0}, sort=[("email", 1)], limit=0):

    collection.create_index(sort)

    return list(collection.find(query, filter).sort(sort).limit(limit))

def update_client(client):

    return collection.update_one({"email": client["email"]}, {"$set": client})

def update_clients(clients):

    for client in clients:

        update_client(client)

def delete_client(client):

    return collection.delete_one({"email": client["email"]})

def delete_clients(clients):

    for client in clients:

        delete_client(client)

def valid_client(id):

    try:

        return find_client({"id": id}, {"_id": 0, "id": 0, "password": 0})

    except:

        return None